In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/myanimelist-dataset/user-filtered.csv
/kaggle/input/myanimelist-dataset/users-score-2023.csv
/kaggle/input/myanimelist-dataset/anime-filtered.csv
/kaggle/input/myanimelist-dataset/anime-dataset-2023.csv
/kaggle/input/myanimelist-dataset/final_animedataset.csv
/kaggle/input/myanimelist-dataset/users-details-2023.csv


# Introduction

    The main objective to achive, it's to use NLP techniques along with linear algebra concepts (cousine similarity),to get anime recommendations.

In [54]:
#additional library importation
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob
import re
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
#importing the dataset
df = pd.read_csv('/kaggle/input/myanimelist-dataset/anime-dataset-2023.csv')

In [88]:
#visualizing the dataset
df.head(3)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...


In [89]:
df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

# **First Step: Pre-Processing the Data**

In [90]:
#extracting only relevant columns for our project
x = df[['Name','Synopsis']]

x.head(3)

,Name,Synopsis
0,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ..."
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o..."
2,Trigun,"Vash the Stampede is the man with a $$60,000,0..."


In [91]:
#counting missing values on the columns
x.isnull().sum()

Name        0
Synopsis    0
dtype: int64

In [92]:
#verifying the existence of duplicate values
x.duplicated().sum()

0

# **Second Step: Applying NLP Techniques**

In [95]:
#defining usefull functions

#function used to remove the string punctuation an turn all into lower case
def remove_punctuation_and_turn_lower(text):
    translator = str.maketrans('', '', string.punctuation)
    return (text.translate(translator)).lower()

#function used to treat the string list to the correct form(no punctuation or numbers or empty spaces,only stemmed strings in their lower case form)
def string_cleaning(string_list):
  strings_limpas = [remove_punctuation_and_turn_lower(string) for string in string_list]

  strings_limpas_no_numbers = [re.sub(r'\d', '', string) for string in strings_limpas]
    
  new_list = [item for item in strings_limpas_no_numbers if item]

  tokens_without_sw = [word for word in new_list if not word in stopwords.words('english')]

  ps = PorterStemmer()
  steemed_words = [ps.stem(w) for w in tokens_without_sw]


  return steemed_words

In [96]:
#treating the synopsis column:

x['Synopsis'] = x['Synopsis'].apply(lambda x: x.split())
x['Synopsis'] = x['Synopsis'].apply(string_cleaning)

x.head(3)

,Name,Synopsis
0,Cowboy Bebop,"[crime, timeless, year, human, expand, across,..."
1,Cowboy Bebop: Tengoku no Tobira,"[anoth, day, anoth, bounty—such, life, often, ..."
2,Trigun,"[vash, stamped, man, bounti, head, reason, he,..."


In [97]:
#applying bag of words technique
x_bow = x['Synopsis'].apply(lambda x: ' '.join(x))

# **Third Step: Verifying cosine similarity at all the resultant vectors**

In [98]:
# Using CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,1), max_features=5000)

new_column = (vectorizer.fit_transform(x_bow)).toarray()

# Calculation of cosine similarity between the vectors
similarities = cosine_similarity(new_column)

In [99]:
print(similarities)

[[1.         0.15841883 0.08708597 ... 0.         0.         0.        ]
 [0.15841883 1.         0.08277634 ... 0.         0.         0.        ]
 [0.08708597 0.08277634 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         1.        ]
 [0.         0.         0.         ... 0.         1.         1.        ]]


In [104]:
def get_anime_recomendation(name):
    #getting the index of the anime on the dataframe
    index = (df.index[df['Name'] == name])[0]
    
    #separating the original list and the ordenated list
    normal_list = similarities[index]
    ordenated_list = sorted(similarities[index],reverse=True)
    
    ranking_list = []
    for i in range(len(ordenated_list)):
        index_new = np.where(normal_list == ordenated_list[i])[0]
        tuple_anime = (int(index_new[0]),ordenated_list[i]) 
        ranking_list.append(tuple_anime)
        
    #getting top 5 anime recommendations: 
    print('Top 5 recommendations based on the selected anime:\n')
    for tuple_values in ranking_list[1:6]:
        print(x.iloc[tuple_values[0]]['Name'])

# **Final Step: TESTING**

In [114]:
get_anime_recomendation('Death Note')

Top 5 recommendations based on the selected anime:

Death Note: Rewrite
Sekaikei Sekai Ron
Ghost Messenger
Wan Jie Shen Zhu
Shinigami no Ballad.
